## Importing Dependencies

In [1]:
# Dependencies
import csv as csv
import geopy.distance
import gmaps
import json
import pandas as pd
import re
import requests
import time
from pprint import pprint
from jsonmerge import Merger
# Google developer API key

## Reading in .csv's generated by MigratioFlowQuery.ipynb

In [2]:
# Reading in the DataFrame that holds the names of all of the counties and states from which the most people 
# immigrated to the target counties from
combined_counties_df = pd.read_csv("../Resources/migration_flow_csvs/combined_county_df.csv")
counts_by_state_df = pd.read_csv("../Resources/migration_flow_csvs/counts_by_state_df.csv")

In [3]:
# Removing Kentucky and Wyoming from counts_by_state_df. These states appear due to the addition of Kern County,
# Collin County, and Wayne County as test counties for later statistical analysis 
counts_by_state_df = counts_by_state_df.loc[(counts_by_state_df["Origin State"] != "Kentucky") &
                                            (counts_by_state_df["Origin State"] != "Wyoming")]
# Initializing a list with the names of each origin state so they can be used to query the center coordinates of each
# origin state
state_list = counts_by_state_df["Origin State"].to_list()

## Defining a function to query the Google Places API

In [4]:
# Defining a function that will take two parameters, "state" and "token" and and query the Google Places ASI
def google_places_api_query(state):
    # Initializing variables with the initial conditions of the query
    state_response_list = []
    token_presence = "yes"
    token = ""
    target_type = "university"
    # Base-url
    base_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
    # Using a while-loop to continue querying the Google Places API until the result limit is reached
    while token_presence == "yes":
        # Initializing a dictionary with the parameters for the query url
        params = {
        #"type": target_type,
        "key": google_key,
        "query": f"Best universities in {state}",
        "pagetoken": token
        }
        # Initializing a variable to hold the query response so that the query url can be displayed
        response = requests.get(base_url, params=params)
        # Initializing a variable to hold the query response in JSON format
        state_response = requests.get(base_url, params=params).json()
        # Using a conditional to direct the function based on the presence of a next_page_token
        if "next_page_token" in state_response:
            token_presence = "yes"
            token = state_response["next_page_token"]
            time.sleep(3)
            
        else:
            token_presence = "no"
        # Appending the query response(s) to state_response_list
        state_response_list.append(state_response)
    # Returning the state_response_list
    return state_response_list

## Querying the Google Places API and merging the results into one JSON object

In [5]:
# Initializing empty dictionary to hold the name of each state as a key and its related Gooogle Places API response 
# as its value
state_response_dict = {}
#
for state in state_list:
    state_response_dict.update({state: google_places_api_query(state)})

In [6]:
# Initializing an empty dictionary to hold the combined json responses for each state
combined_value_dict = {}
# Using a for loop to combine the multiple json responses for each state
for state in state_list:
    # Initializing a variable to hold the range of the values assigned to the key of each state name
    state_values = range(len(state_response_dict[state]))
    # Initializing an initial condition for the base that the first json response will merge with
    base = None    
    # Using a for loop to initialize a schema upon which the json responses will be merged and merging them
    for i in state_values:
        # Initializing the schema upon which the json responses will be merged
        schema = {
                     "properties": {
                         "results": {
                             "type": "object",
                             "mergeStrategy": "arrayMergeById"
                         }
                     }
                 }
        # Initializing a variable to hold the schema after Merger
        merger = Merger(schema)
        # Initializing a varaibel to hold the response the loop is currently working on
        v1 = state_response_dict[state][i]
        # Updating base to hold the merge of the base and v1
        base = merger.merge(base, v1)
    # Updating combined_value_dict
    combined_value_dict.update({state: base})
# Writing a .json file to hold the entire dictionary of merged json responses
with open("../Resources/jsons/places_query_result.json", 'w') as write_file:
    json.dump(combined_value_dict, write_file)

## Optional code to read in the text file containing the merged JSON object so further queries do not have to be made.

In [4]:
# This is an optional piece of code that can be run in cases where the Google Places API query has already been 
# carried out and the notebook has been re-opened, so that further blocks of code can be run without having to 
# re-query the API
with open("../Resources/jsons/places_query_result.json", "r") as read_file:
    combined_value_dict = json.load(read_file)

## Assembling components for future DataFrames

In [7]:
# Initializing an empty dictionary that will hold each county_state as a key and a list of the schools in that 
# state as its value
state_school_dict = {}
state_school_lat_lng_df = pd.DataFrame()
school_name_list = []
ind = 0
# Using a for-loop to append schools in each county_state to school_list and then to save school_list as a value to
# each county_state
for state in state_list:
    state_school_list = []
    # Using a for-loop to assign the values retrieved from combined_value_dict and assign them to variables
    try:
        for i in range(len(combined_value_dict[state]["results"])):
            state_school_name = combined_value_dict[state]["results"][i]["name"]
            state_school_lat = combined_value_dict[state]["results"][i]["geometry"]["location"]["lat"]
            state_school_lng = combined_value_dict[state]["results"][i]["geometry"]["location"]["lng"]
            state_school_list.append(state_school_name)
            # Initializing a temporary DataFrame to hold the collected values
            state_school_lat_lng = pd.DataFrame({"University Name": state_school_name,
                                       "State": state,
                                       "Lat": state_school_lat,
                                       "Lng": state_school_lng},
                                       index=[ind])
            # Appending state_school_lat_lng_df with the temporary DataFrame
            state_school_lat_lng_df = state_school_lat_lng_df.append(state_school_lat_lng)
            # Appending the name of the school the loop was working on to school_name_list
            school_name_list.append(state_school_name)
            # Incrementing ind for the next iteration of the loop
            ind += 1
            # Updating state_school_dict with list of collected 
            state_school_dict.update({state: state_school_list})
    except KeyError:
        pass

In [8]:
# Adding the names of universities in the origin states that did not appear in the query results
school_name_list.append("University of Massachusetts")
school_name_list.append("Carnegie Mellon University")
school_name_list.append("Pennsylvania State University")
school_name_list.append("Stony Brook University")
school_name_list.append("Syracuse University")
# Initializing a DataFrame to hold all of the school names in school_name_list
school_name_df = pd.DataFrame(school_name_list, columns=["University Name"])

In [9]:
# Initialiing a DataFrame with the missing information for the universities in the origin states that did not 
# appear in the query results
missing_school_info_df = pd.DataFrame({"University Name": ["University of Massachusetts",
                                                           "Carnegie Mellon University",
                                                           "Pennsylvania State University",
                                                           "Stony Brook University",
                                                           "Syracuse University"],
                                       "State": ["Massachusetts",
                                                 "Pennsylvania", 
                                                 "Pennsylvania", 
                                                 "New York", 
                                                 "New York"],
                                       "Lat": [42.391155, 40.443322, 39.953885, 40.914224, 43.03767],
                                       "Lng": [-72.526711, -79.943583, -75.193048, -73.11623, -76.13399]})
# Appending missing_school_info_df with state_school_lat_lng_df so it can later be merged the final DataFrame
state_school_lat_lng_df = state_school_lat_lng_df.append(missing_school_info_df, ignore_index = True)

In [10]:
# Reading in the Times csv of the top ranked universities in the world
top_schools_df = pd.read_csv("../Resources/world-university-rankings/timesData.csv")
# Dropping any schools from top_schools_df that are not in the United States of America
top_schools_df = top_schools_df[top_schools_df["country"] == "United States of America"]
# Formating the columns of the top_schools_df to be in title format
top_schools_df.columns = [x.title() for x in top_schools_df.columns]
# Formating the columns of the top_schools_df to no longer have "_" in their names
top_schools_df.columns = top_schools_df.columns.str.replace('_', ' ')
# Initializing a DataFrame to hold the names of the universities 
top_schools_names_df = top_schools_df["University Name"].to_frame()
# Initializing a list to hold the the names of the universities 
top_schools_names_list = top_schools_df["University Name"].to_list()

In [11]:
# Iterating through the top_schools_df and dropping rows that have values in the "World Rank" column that are not 
# discrete numbers
for index, row in top_schools_df.iterrows():
    if "-" in row["World Rank"] or "=" in row["World Rank"]:
        top_schools_df.drop(index, inplace=True)
# Changing the type of the values held ing "World Rank" to floats so that their mean can be taken
top_schools_df["World Rank"] = top_schools_df["World Rank"].astype(float)
# Adding a column named "State" to the top_schools_df
top_schools_df["State"] = ""

## Creating a DataFrame that holds data about each of the top ranked schools in each of the origin states

In [13]:
# Initializing a new column in school_name_df to eventually hold the name of the state each school is in
school_name_df["State"] = "yes"
# Initializing a DataFrame with the mean of "World Rank" taken for each university
world_rank_mean_df = top_schools_df.groupby(["University Name"]).mean()
# Merging cleaned_school_name_df and world_rank_mean_df to create a DataFrame called name_rank_df
name_rank_df = pd.merge(school_name_df, world_rank_mean_df, on="University Name")
# Merging name_rank_df and state_school_lat_lng_df to create a DataFrame called name_rank_coord_df
name_rank_latlng_df = pd.merge(name_rank_df, state_school_lat_lng_df, on="University Name", how="left")
# Deleting the "State_x" column created during the merge and renaming "State_y" to "State"
cleaned_school_df = name_rank_latlng_df.drop("State_x",axis=1).rename(index=str, columns={"State_y": "State"})
# Dropping duplicate rows
cleaned_school_df = cleaned_school_df.drop_duplicates("University Name").sort_values("World Rank").reset_index()
# Droping the "index" column
cleaned_school_df = cleaned_school_df.drop("index",axis=1)
# Creating a column with each school's points
cleaned_school_df["Points"] = 191 - cleaned_school_df["World Rank"]
# Displaying cleaned_school_df
display(cleaned_school_df)

,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points
0,Harvard University,2.833333,93.700000,98.466667,99.266667,8.9,2013.500000,Massachusetts,42.377003,-71.116660,188.166667
1,Massachusetts Institute of Technology,5.166667,92.466667,88.966667,99.916667,9.0,2013.500000,Massachusetts,42.360091,-71.094160,185.833333
2,Princeton University,6.000000,88.916667,96.350000,99.716667,8.4,2013.500000,New Jersey,40.343094,-74.655074,185.000000
3,Yale University,10.666667,90.350000,90.416667,94.616667,4.4,2013.500000,Connecticut,41.316324,-72.922343,180.333333
4,Columbia University,14.333333,84.833333,79.716667,95.916667,5.9,2013.500000,New York,40.807536,-73.962573,176.666667
5,University of Pennsylvania,16.500000,81.600000,84.716667,95.950000,6.5,2013.500000,Pennsylvania,39.952219,-75.193214,174.500000
6,University of Michigan,18.166667,76.533333,88.066667,90.683333,9.0,2013.500000,Michigan,42.278044,-83.738224,172.833333
7,Carnegie Mellon University,22.166667,66.650000,81.150000,95.600000,13.1,2013.500000,Pennsylvania,40.443322,-79.943583,168.833333
8,University of Wisconsin-Madison,33.400000,70.600000,72.920000,88.060000,10.8,2014.000000,Wisconsin,43.076592,-89.412487,157.600000
9,New York University,42.166667,67.400000,62.766667,89.516667,6.8,2013.500000,New York,40.729513,-73.996461,148.833333


## Creating DataFrames for each state that holds data about each of the top ranked schools in that state

In [18]:
# Initializing an empty dictionary to hold the school information grouped by state
school_data_by_state_dict = {}
state_total_school_points = []
# Using a for-loop to assign school information as values to each state as a key
for state in state_list:
    state_df = cleaned_school_df.loc[cleaned_school_df["State"] == state, :]
    school_data_by_state_dict.update({state: state_df})
    print(state)
    # Displaying each state's DataFrame
    display(school_data_by_state_dict[state])
    state_total_school_points.append([state, sum(school_data_by_state_dict[state]["Points"])])
    # Creating a csv for each of the user-defined query_counties
    state_df.to_csv(f"../Resources/state_schools_csvs/{state}_schools_by_state.csv", index=False, header=True)

Michigan


,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points
6,University of Michigan,18.166667,76.533333,88.066667,90.683333,9.0,2013.5,Michigan,42.278044,-83.738224,172.833333
16,Michigan State University,95.400000,50.140000,51.520000,71.320000,15.7,2013.0,Michigan,42.701848,-84.482172,95.600000


Wisconsin


,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points
8,University of Wisconsin-Madison,33.4,70.6,72.92,88.06,10.8,2014.0,Wisconsin,43.076592,-89.412487,157.6


Pennsylvania


...

,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points


New Hampshire


,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points


Maine


,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng,Points


In [23]:
state_total_school_points_df = pd.DataFrame(state_total_school_points, columns=["State", "Total Points"])
state_total_school_points_df.to_csv(f"../Resources/state_schools_csvs/state_total_school_points_df.csv", index=False, header=True)

In [12]:
# Initializing a DataFrame to hold the top school in each state
top_school_per_state_df = pd.DataFrame(columns = ['University Name', 'World Rank', 'Teaching', 'Research', 'Citations',
       'Student Staff Ratio', 'Year', 'State', 'Lat', 'Lng'])
# Using a for-loop ro retrieve the top rated school in each state and assign it as a value in a dictionary
for school in school_data_by_state_dict:
    # Retrieving the top school in each state
    top_school = school_data_by_state_dict[school].iloc[0]
    # Appending the top shcool to top_school_per_state_df 
    top_school_per_state_df = top_school_per_state_df.append(top_school) 
# Exporting top_school_per_state_df as a csv
top_school_per_state_df.to_csv(f"../Resources/state_schools_csvs/top_schools_per_state.csv", index=False, header=True)
# Displaying top_school_per_state_df
display(top_school_per_state_df)

,University Name,World Rank,Teaching,Research,Citations,Student Staff Ratio,Year,State,Lat,Lng
0,Harvard University,2.833333,93.700000,98.466667,99.266667,8.9,2013.5,Massachusetts,42.377003,-71.116660
6,University of Michigan,18.166667,76.533333,88.066667,90.683333,9.0,2013.5,Michigan,42.278044,-83.738224
5,University of Pennsylvania,16.500000,81.600000,84.716667,95.950000,6.5,2013.5,Pennsylvania,39.952219,-75.193214
4,Columbia University,14.333333,84.833333,79.716667,95.916667,5.9,2013.5,New York,40.807536,-73.962573
2,Princeton University,6.000000,88.916667,96.350000,99.716667,8.4,2013.5,New Jersey,40.343094,-74.655074
3,Yale University,10.666667,90.350000,90.416667,94.616667,4.4,2013.5,Connecticut,41.316324,-72.922343
10,University of Minnesota,46.600000,62.220000,68.600000,82.120000,18.0,2013.0,Minnesota,44.973990,-93.227728
8,University of Wisconsin-Madison,33.400000,70.600000,72.920000,88.060000,10.8,2014.0,Wisconsin,43.076592,-89.412487


## Creating a heat map that displays the locations of the highest ranked schools in each state weighted by their world rank

In [13]:
location = cleaned_school_df[["Lat", "Lng"]]
rating = cleaned_school_df["World Rank"]
        
fig = gmaps.figure()
# Create heat layer
heat_layer = gmaps.heatmap_layer(location, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.1)
# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))